# (Dynamic) Multiple Dispatch in Julia

[Multiple dispatch](http://en.wikipedia.org/wiki/Multiple_dispatch) can be thought of as a generalization of **object-oriented** (OO) programming.

In a typical OO language like Python, an object type (class) *owns* certain *methods* (functions), and are typically called via
```
object.method(arg1, arg2)
```
Depending on the type of `object`, the runtime system will *dispatch* to different `method` definitions.

In Julia, the same call would be "spelled" differently:
```
method(object, arg1, arg2)
```
Spelled this way, you should notice something odd about OO programming: why is the *first* argument so special?

Traditional OO programming corresponds to **single dispatch**: the runtime chooses `method` based on the type of the *first* argument only.   Julia implements **multiple dispatch**: the runtime chooses `method` based on the types of *all* the arguments.

## Example: Binary mathematical operators

A classic example of the need for multiple dispatch is the case of binary math operators.  If you compute `x * y`, the definition of the `*` function depends upon *both* the arguments, not just on `x`.

Julia defines *many versions* of the `*` function:

In [1]:
methods(*)

# 149 methods for generic function "*":
*(x::Bool, z::Complex{Bool}) at complex.jl:140
*(x::Bool, y::Bool) at bool.jl:50
*{T<:Unsigned}(x::Bool, y::T) at bool.jl:63
*(x::Bool, z::Complex) at complex.jl:147
*(x::Bool, y::Irrational) at irrationals.jl:90
*{T<:Number}(x::Bool, y::T) at bool.jl:60
*(x::Float32, y::Float32) at float.jl:243
*(x::Float64, y::Float64) at float.jl:244
*(z::Complex{Bool}, x::Bool) at complex.jl:141
*(z::Complex{Bool}, x::Real) at complex.jl:159
*(a::Float16, b::Float16) at float16.jl:136
*(this::Base.Grisu.Float, other::Base.Grisu.Float) at grisu/float.jl:138
*(c::BigInt, x::BigFloat) at mpfr.jl:240
*(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:298
*(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:291
*(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:285
*(x::BigInt, y::BigInt) at gmp.jl:255
*(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:332
*(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:338
*(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:388
*(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:381
*(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:375
*(x::BigFloat, c::BigInt) at mpfr.jl:236
*(x::BigFloat, y::BigFloat) at mpfr.jl:205
*(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:212
*(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:220
*(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:228
*(B::BitArray{2}, J::UniformScaling) at linalg/uniformscaling.jl:132
*(z::Complex, w::Complex) at complex.jl:127
*{T<:Union{Int128,UInt128}}(x::T, y::T) at int.jl:426
*{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(x::T, y::T) at int.jl:33
*(z::Complex, x::Bool) at complex.jl:148
*(x::Real, z::Complex{Bool}) at complex.jl:158
*(x::Real, z::Complex) at complex.jl:170
*(z::Complex, x::Real) at complex.jl:171
*(x::Rational, y::Rational) at rational.jl:206
*{N}(a::Integer, index::CartesianIndex{N}) at multidimensional.jl:61
*(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:336
*(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:342
*(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:216
*(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:224
*(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:232
*{T<:Number}(x::T, D::Diagonal) at linalg/diagonal.jl:113
*(x::Irrational, y::Irrational) at irrationals.jl:88
*(y::Real, x::Base.Dates.Period) at dates/periods.jl:90
*(x::Number) at operators.jl:116
*(y::Number, x::Bool) at bool.jl:65
*{T<:Number}(x::T, y::T) at promotion.jl:256
*(x::Number, y::Number) at promotion.jl:191
*{T<:Union{Complex{Float32},Complex{Float64},Float32,Float64},S}(A::Union{Base.ReshapedArray{T,2,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{T,2},SubArray{T,2,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}, x::Union{Base.ReshapedArray{S,1,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray{S,1},SubArray{S,1,A<:Union{Base.ReshapedArray{T<:Any,N<:Any,A<:DenseArray,MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N<:Any}}},DenseArray},I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex,Colon,Int64,Range{Int64}},N<:Any}},L<:Any}}) at linalg/matmul.jl:79
*(A::SymTridiagonal, B::Number) at linalg/tridiag.jl:98
*(A::Tridiagonal, B::Number) at linalg/tridiag.jl:496
*(A::UpperTriangular, x::Number) at linalg/triangular.jl:477
*(A::Base.LinAlg.UnitUpperTriangular, x::Number) at linalg/triangular.jl:480
*(A::LowerTriangular, x::Number) at linalg/triangular.jl:477
*(A::Base.LinAlg.UnitLowerTriangular, x::Number) at linalg/triangular.jl:480
*(A::Tridiagonal, B::Up

We can add new methods to a given function at any time.  The methods don't "belong" to a particular type, and aren't part of the type's definition.

For example, string concatenation in Julia is done via `*`:

In [2]:
"hello" * "world"

"helloworld"

In [3]:
"hello" + "world"

LoadError: LoadError: MethodError: no method matching +(::String, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
while loading In[3], in expression starting on line 1

But we can easily extend `+` to support a concatenation for strings, if we want:

In [4]:
import Base.+ # we must import a method to add methods (as opposed to replacing it)
+(x::String, y::String) = x * " " * y

+ (generic function with 164 methods)

In [5]:
"hello" + "world"

"hello world"

## Not the same as C++ overloading: Dynamic vs. static

This may look a lot like function overloading in languages like C++.   The difference is that C++'s overloading is **static** (= *dispatch at compile-time*), whereas Julia's overloading is dynamic (= *dispatch at run-time*), like OO polymorphism.

For example, now that we've defined `+`, we can use strings with any previously defined function that requires a `+` operation, like `sum` (summation):

In [6]:
sum(["The", "quick", "brown", "fox", "jumped", "over", "the", "lazy", "dog."])

"The quick brown fox jumped over the lazy dog."

## Type declarations are "function filters"

Type declarations are *not* required for performance — Julia automatically specializes a function on its argument types during compilation.  They act like **filters**, allowing us to specify *which functions are used when*.

Without this, in a language like Python, you sometimes have to write *manual function filters* (see, for example, `_parse_args(*args)` in Matplotlib's [quiver.py](https://github.com/matplotlib/matplotlib/blob/43150ef012e405de5c2fb7b779d1bffa77f7af81/lib/matplotlib/quiver.py)).

In Julia, you could define different methods for differing numbers of arguments, arrays vs. scalars, etcetera (all eventually calling a single lower-level function to do the work once the arguments have been transformed).

The work is done by the assertion operator `::`, as seen below:

In [7]:
(1+2)::AbstractFloat

LoadError: LoadError: TypeError: typeassert: expected AbstractFloat, got Int64
while loading In[7], in expression starting on line 1

In [8]:
(1+2)::Int

3

Julia works with a dynamic tree of types:

In [9]:
Integer <: Number

true

This allows type assertions attached in-place, both in in-line code and in function declarations. They are useful to structure and describe the logical flow of your program, as well as enabling some performance improvements from the compiler. A type assertion for returned values is also possible; here is an example that gives some of the flavour:

In [10]:
function clip{T<:Real}(x::T, lo::Real, hi::Real)::T
    if x < lo
        return lo
    elseif x > hi
        return hi
    else
        return x
    end
end

clip (generic function with 1 method)

In [11]:
clip(0.5, 1, 2) # converts lo to T

1.0

In [12]:
clip(1.5, 1, 2)

1.5

In [13]:
clip(5//2, 1, 2) # converts hi to T

2//1